In [73]:
import fitz  # PyMuPDF
from collections import defaultdict
import json
import base64

## convert to json

In [22]:
# Open the PDF file
doc = fitz.open("The Digital Code of DNA.pdf")  # Replace with the actual PDF file path
raw_data = []

# Process each page in the document
for page in doc:
    print(page)
    blocks = page.get_text("dict")
    raw_data.append(blocks)
    # for block in blocks: raw_data.append(block)

page 0 of The Digital Code of DNA.pdf
page 1 of The Digital Code of DNA.pdf
page 2 of The Digital Code of DNA.pdf
page 3 of The Digital Code of DNA.pdf
page 4 of The Digital Code of DNA.pdf


### add id

In [91]:
id_counter = 0
for page in raw_data:
  for block in page["blocks"]:
    if 'lines' in block:
      for line in block["lines"]:
        for span in line["spans"]:
          span["id"] = id_counter
          id_counter+=1
    elif 'image' in block:
      block["id"] = id_counter
      id_counter+=1

In [92]:
# Convert object data to JSON
json_data = json.dumps(raw_data, default=str)

with open('./raw_pdf.json', 'w') as json_file:
    json_file.write(json_data)

## Convert to html

In [276]:
def process_block(element):
    spans = []
    if 'lines' in element:
        for line in element['lines']:

            # decide for writing-mode
            writing_mode = 'horizontal-tb'
            if line["dir"][1] == -1: writing_mode = 'vertical-lr'

            for span in line['spans']:
                span["writing_mode"] = writing_mode
                spans.append(span)
    elif 'image' in element:
        spans.append(element)
    else: print(element)

    return spans

# Open the HTML file in write mode
with open('recreated.html', 'w') as html_file:
    # Write the HTML file header
    html_file.write("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Output HTML</title>
        <style>
            .divhoverable:hover {background-color: red}
            .spanhoverable:hover {background-color: green}
        </style>
    </head>
    <body>
    """)

    # Process each element in the data array
    page_width = raw_data[0]["width"]
    page_height = raw_data[0]["height"]
    page_index = -1

    for page in raw_data:
        page_index += 1
        # Process the element
        for block in page["blocks"]:
            
            if 'lines' in block:
                html_file.write(f"""<div
                    class="divhoverable"
                    style="
                        position: absolute;
                        left: {block["bbox"][0]}px;
                        top: {block["bbox"][1]+(page_index*page_height)}px;
                        width: {block["bbox"][2]-block["bbox"][0]+6}px;
                        height: {block["bbox"][3]-block["bbox"][1]}px;
                        line-height: 6px;
                    "
                >""")

                for line in block['lines']:

                    # decide for writing-mode
                    writing_mode = 'horizontal-tb'
                    if line["dir"][1] == -1: writing_mode = 'vertical-lr'

                    for span in line['spans']:
                        span["writing_mode"] = writing_mode

                        tag_name = 'span'
                        # if span["flags"] == 20: tag_name = 'p'
                        # Generate HTML content for the processed element
                        html_content = f"""
                        <{tag_name}
                            class="spanhoverable"
                            page_index="{page_index}"
                            style="
                                text-align: justify;
                                font-size: {span["size"]-.5}px;
                                line-height: 1.1;
                                writing-mode: {span["writing_mode"]};
                                color: rgb({(span["color"] >> 16) & 255}, {(span["color"] >> 8) & 255}, {span["color"] & 255});
                            "
                        >{span["text"]}</{tag_name}>
                        """

                        # Write the HTML content to the file
                        html_file.write(html_content)
                
                html_file.write(f"""</div>""")

            else:
                base64_image = block["image"]
                image_data_str = base64.b64encode(base64_image).decode('utf-8')

                image_data_html = f"data:image/jpeg;base64,{image_data_str}"

                html_content = f"""
                <img
                    page_index="{page_index}"
                    src="{image_data_html}"
                    width="{block["bbox"][2]-block["bbox"][0]}"
                    height="{block["bbox"][3]-block["bbox"][1]}"
                    style="
                        position: absolute;
                        left: {block["bbox"][0]}px;
                        top: {block["bbox"][1]+(page_index*page_height)}px;
                    "
                >
                """
                    #   transform: matrix{block["transform"]}
                html_file.write(html_content)


    # Write the HTML file footer
    html_file.write("""
    </body>
    </html>
    """)

print("HTML file has been generated successfully.")


HTML file has been generated successfully.
